Before we get the data and start exploring it, let's download all the dependencies that we will need.

In [1]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
import requests # library to handle requests
from bs4 import BeautifulSoup #library for parsing html code
# import k-means from clustering stage
from sklearn.cluster import KMeans


#### Load and explore the data

Create a class for parsing table tag with class name in html content and convert table into pandas dataframe

In [57]:
 class HTMLTableParser:
       
        def parse_table(self, url, class_name = ''):
            response = requests.get(url)
            soup = BeautifulSoup(response.text, 'lxml')
            table = soup.find('table', class_ = class_name)
            return self.parse_html_table(table)
    
        def parse_html_table(self, table):
            n_columns = 0
            n_rows=0
            column_names = []
    
            # Find number of rows and columns
            # we also find the column titles if we can
            for row in table.find_all('tr'):
                
                # Determine the number of rows in the table
                td_tags = row.find_all('td')
                if len(td_tags) > 0:
                    n_rows+=1
                    if n_columns == 0:
                        # Set the number of columns for our table
                        n_columns = len(td_tags)
                        
                # Handle column names if we find them
                th_tags = row.find_all('th') 
                if len(th_tags) > 0 and len(column_names) == 0:
                    for th in th_tags:
                        column_names.append(th.get_text().rstrip())
    
            # Safeguard on Column Titles
            if len(column_names) > 0 and len(column_names) != n_columns:
                raise Exception("Column titles do not match the number of columns")
    
            columns = column_names if len(column_names) > 0 else range(0,n_columns)
            df = pd.DataFrame(columns = columns,
                              index= range(0,n_rows))
            row_marker = 0
            for row in table.find_all('tr'):
                column_marker = 0
                columns = row.find_all('td')
                for column in columns:
                    df.iat[row_marker,column_marker] = column.get_text().rstrip()
                    column_marker += 1
                if len(columns) > 0:
                    row_marker += 1            
         
            return df

Now, we use HTMLTableParser to parse postal codes table from url https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M </br>
**Note:** The html class to indentify the table is "wikitable"        

In [66]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
hp = HTMLTableParser()
wiki_df = hp.parse_table(url, 'wikitable')
wiki_df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


Drop all rows that their borough is 'Not assigned'

In [67]:
wiki_df = wiki_df.drop(wiki_df[wiki_df.Borough == 'Not assigned'].index)
wiki_df.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


In [68]:
print(wiki_df.loc[wiki_df.Neighbourhood == 'Not assigned'])

  Postcode       Borough Neighbourhood
8      M7A  Queen's Park  Not assigned


If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park

In [72]:
wiki_df.Neighbourhood = wiki_df.Neighbourhood.replace('Not assigned', wiki_df.Borough)
print(wiki_df.loc[wiki_df.Postcode == 'M7A'])

  Postcode       Borough Neighbourhood
8      M7A  Queen's Park  Queen's Park


More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

In [73]:
wiki_df = wiki_df.groupby(['Postcode','Borough']).agg( ','.join).reset_index()
wiki_df.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [74]:
print(wiki_df.shape[0])

103
